In [ ]:
import opsimsummaryv2 as op

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys
from astropy.cosmology import FlatLambdaCDM
from astropy.units import Quantity
from slsim.lens_pop import LensPop
from slsim.image_simulation import (
    sharp_image,
    sharp_rgb_image,
    rgb_image_from_image_list,
)
import galsim
import mpl_toolkits.axisartist.floating_axes as floating_axes
from mpl_toolkits.axisartist.grid_finder import MaxNLocator, DictFormatter
from matplotlib.transforms import Affine2D
# import lsst.daf.butler as dafButler
# import lsst.geom as geom
# import lsst.afw.display as afwDisplay
# from lsst.pipe.tasks.insertFakes import _add_fake_sources
from slsim import lsst_science_pipeline
import astropy.coordinates as coord
import astropy.units as u

In [ ]:
print("done")

In [ ]:
# Import OpSimSummaryV2
try:
    import opsimsummaryv2 as op
except ImportError:
    raise ImportError(
        "Users need to have OpSimSummaryV2 installed (https://github.com/bastiencarreres/OpSimSummaryV2)"
    )

# Initialise OpSimSummaryV2 with opsim database
try:
    opsim_path = "../data/OpSim_database/" + obs_strategy + ".db"
    OpSimSurv = op.OpSimSurvey(opsim_path)
except FileNotFoundError:
    raise FileNotFoundError(
        "File not found: " + opsim_path + ". Input variable 'obs_strategy' should correspond to the name of an opsim database saved in the folder ../data/OpSim_database"
    )

# Collect observations that cover the coordinates in ra_list and dec_list
gen = OpSimSurv.get_obs_from_coords(ra_list, dec_list, is_deg=True)

In [ ]:
N = 10

ra_points = coord.Angle(np.random.uniform(low=0, high=360, size=N) * u.degree)
ra_points = ra_points.wrap_at(180*u.degree)
dec_points = np.arcsin(2 * np.random.uniform(size=N) - 1) / np.pi * 180
dec_points = coord.Angle(dec_points * u.degree)

In [ ]:
exposure_data = lsst_science_pipeline.opsim_time_series_images_data(ra_points, dec_points, "baseline_v3.0_10yrs",
                                                                    MJD_min=60000, MJD_max=60300)
exposure_data

In [ ]:
# add galsim to requirements
# colossus is listed twice

## Lensed source injection in DC2 data

This notebook uses slsim to generate lens-deflector population. Then, we select a random lens-deflector

and inject it to a patch of the DC2 data.

## Generate population of sources and deflectors

Using slsim one can generate galaxy-galaxy lenses.

In [ ]:
## Users should change this path to their slsim path
sys.path.insert(0, "../slsim/")

In [ ]:
# define a cosmology
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# define a sky area
sky_area = Quantity(value=0.1, unit="deg2")


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

# run skypy pipeline and make galaxy-galaxy population class using LensPop
gg_lens_pop = LensPop(
    deflector_type="all-galaxies",
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config=None,
    sky_area=sky_area,
    cosmo=cosmo,
)

## Select a lens at random and generate a high resolution image

In [ ]:
kwargs_lens_cut = {
    "min_image_separation": 3,
    "max_image_separation": 10,
    "mag_arc_limit": {"g": 22, "r": 22, "i": 22},
}
rgb_band_list = ["i", "r", "g"]
lens_class = gg_lens_pop.select_lens_at_random(**kwargs_lens_cut)



In [ ]:
image_i_1 = sharp_image(
    lens_class=lens_class,
    band=rgb_band_list[0],
    mag_zero_point=27,
    delta_pix=0.2,
    num_pix=200,
)
image_r_1 = sharp_image(
    lens_class=lens_class,
    band=rgb_band_list[1],
    mag_zero_point=27,
    delta_pix=0.2,
    num_pix=200,
)
image_g_1 = sharp_image(
    lens_class=lens_class,
    band=rgb_band_list[2],
    mag_zero_point=27,
    delta_pix=0.2,
    num_pix=200,
)

In [ ]:
# image_g.shape
plt.imshow(image_i_1, origin="lower")
plt.xlim(75, 125)
plt.ylim(75, 125)

In [ ]:
high_reso_rgb = sharp_rgb_image(
    lens_class=lens_class,
    rgb_band_list=rgb_band_list,
    mag_zero_point=27,
    delta_pix=0.2,
    num_pix=200,
)

plt.imshow(high_reso_rgb, origin="lower")
plt.xlim(75, 125)
plt.ylim(75, 125)

## Inject the randomly selected lens 

In [ ]:
ra = 62.541629  # degrees
dec = -37.852021  # degrees

ra_list = [ra]
dec_list = [dec]
delta_pix = 0.2

In [ ]:
import opsimsummary

Replace np.int by int 
~/anaconda3/envs/slsim/lib/python3.12/site-packages/opsimsummary/summarize_opsim.py

In [ ]:
y = lsst_science_pipeline.lens_inejection(
    gg_lens_pop, 201, 0.2, butler, ra, dec, flux=None
)

In [ ]:
## This line should display an astropy table containg lens image,dp0 cutout_image, injected_lens
## in r, g, and i band and  center of the dp0 cutout images.
y

In [ ]:
import lenstronomy.Util.kernel_util as kernel_util
import lenstronomy.Util.util as util
import lenstronomy.Util.data_util as data_util
from astropy.table import Table, vstack

size = 101
delta_pix = 0.2
psf_fwhm = 0.7
moffat_beta = 3.1

# Create a Moffat psf kernel
psf_kernel = kernel_util.kernel_moffat(num_pix=size, delta_pix=delta_pix, fwhm=psf_fwhm, 
                                       moffat_beta=moffat_beta)

psf_kernel = util.array2image(psf_kernel)

psf_kernel

In [ ]:
def opsim_time_series_images_data2(
    ra_list, dec_list, obs_strategy, MJD_min=60000, MJD_max=64000, size=101, moffat_beta=3.1, 
    readout_noise=10, delta_pix=0.2
):
    """Creates time series data from opsim database.

    :param ra_list: a list of ra points (in degrees) from objects we want to collect
        observations for
    :param dec_list: a list of dec points (in degrees) from objects we want to collect
        observations for
    :param obs_strategy: version of observing strategy corresponding to opsim database.
        for example "baseline_v3.0_10yrs" (string)
    :param size: cutout size of images (in pixels)
    :param moffat_beta: power index of the moffat psf kernel
    :param readout_noise: noise added per readout
    :param delta_pix: size of pixel in units arcseonds
    :return: a list of astropy tables containing observation information for each
        coordinate
    """

    # Import OpSimSummaryV2
    try:
        import opsimsummaryv2 as op
    except ImportError:
        raise ImportError(
            "Users need to have OpSimSummaryV2 installed (https://github.com/bastiencarreres/OpSimSummaryV2)"
        )

    # Initialise OpSimSummaryV2 with opsim database
    try:
        opsim_path = "../data/OpSim_database/" + obs_strategy + ".db"
        OpSimSurv = op.OpSimSurvey(opsim_path)
    except FileNotFoundError:
        raise FileNotFoundError(
            "File not found: " + opsim_path + ". Input variable 'obs_strategy' should correspond to the name of an opsim database saved in the folder ../data/OpSim_database"
        )

    # Collect observations that cover the coordinates in ra_list and dec_list
    gen = OpSimSurv.get_obs_from_coords(ra_list, dec_list, is_deg=True)

    table_data_list = []

    # Loop through all coordinates and compute the table_data
    for i in range(len(ra_list)):

        # Collect the next observation sequence from the opsim generator
        seq = next(gen)
        seq = seq.sort_values(by=["observationStartMJD"])

        # Check if the coordinates are in the opsim LSST footprint
        opsim_ra = np.mean(seq["fieldRA"])
        opsim_dec = np.mean(seq["fieldDec"])

        if np.isnan(opsim_ra) or np.isnan(opsim_dec):
            print(
                f"Coordinate ({ra_list[i]}, {dec_list[i]}) is not in the LSST footprint. This entry is skipped."
            )
            continue

        # Get the relevant properties from opsim
        obs_time = np.array(seq["observationStartMJD"])
        
        # Only give the observations between MJD_min and MJD_max
        mask = (obs_time > MJD_min) & (obs_time < MJD_max)
        obs_time = obs_time[mask]
        
        expo_time = np.array(seq["visitExposureTime"])[mask]
        sky_brightness = np.array(seq["skyBrightness"])[mask]
        bandpass = np.array(seq["filter"])[mask]
        psf_fwhm = np.array(seq["seeingFwhmGeom"])[mask]
        m5_depth = np.array(seq["fiveSigmaDepth"])[mask]
        # Question: use 'FWHMeff' or 'seeingFwhmGeom' for the psf?
        
        radec_list = [(ra_list[i], dec_list[i])] * len(obs_time)

        # Create a Moffat psf kernel for each epoch
        
        psf_kernels = []
        
        for psf in psf_fwhm:
        
            psf_kernel = kernel_util.kernel_moffat(
                num_pix=size, delta_pix=delta_pix, fwhm=psf, moffat_beta=moffat_beta
            )
            psf_kernel = util.array2image(psf_kernel)
            
            psf_kernels.append(psf_kernel)
            
        psf_kernels = np.array(psf_kernels)

        # Calculate background noise
        bkg_noise = data_util.bkg_noise(readout_noise, expo_time, sky_brightness, delta_pix, num_exposures=1)

        # Calculate the zero point magnitude
        # Code from OpSimSummary/opsimsummary/simlib.py/add_simlibCols
        # need to work in nvariance in photon electrons
        term1 = 2.0 * m5_depth - sky_brightness  # * pixArea   whata units is sky birghtness? counts or photo electrons?
        # per pixel or arcsec?
        term2 = -(m5_depth - sky_brightness)  # * pixArea
        area = (1.51 * psf_fwhm) ** 2.0  # area = 1 / int(psf^2)
        opsim_snr = 5.0
        arg = area * opsim_snr * opsim_snr
        # Background dominated limit assuming counts with system transmission only
        # is approximately equal to counts with total transmission
        zpt_approx = term1 + 2.5 * np.log10(arg)
        val = -0.4 * term2
        tmp = 10.0**val
        # Additional term to account for photons from the source, again assuming
        # that counts with system transmission approximately equal counts with total transmission.
        zpt_cor = 2.5 * np.log10(1.0 + 1.0 / (area * tmp))
        zero_point_mag = zpt_approx + zpt_cor
        
        table_data = Table(
            [
                bkg_noise,
                psf_kernels,
                obs_time,
                expo_time,
                zero_point_mag,
                radec_list,
                bandpass,
            ],
            names=(
                "bkg_noise",
                "psf_kernel",
                "obs_time",
                "expo_time",
                "zero_point",
                "calexp_center",
                "band",
            ),
        )

        table_data_list.append(table_data)
    return table_data_list

In [ ]:
def opsim_variable_lens_injection(
    lens_class, bands, num_pix, transform_pix2angle, exposure_data
):
    """Injects variable lens to the OpSim time series data (1 object).

    :param lens_class: Lens() object
    :param bands: list of imaging bands of interest
    :param num_pix: number of pixels per axis
    :param transform_pix2angle: transformation matrix (2x2) of pixels into coordinate
        displacements
    :param exposure_data: An astropy table of exposure data. One entry of table_list_data
        generated from the opsim_time_series_images_data function. It must contain the rms of
        background noise fluctuations (column name should be "bkg_noise"), psf kernel for
        each exposure (column name should be "psf_kernel", these are pixel psf kernel
        for each single exposure images in time series image), observation time
        (column name should be "obs_time", these are observation time in days for each
        single exposure images in time series images), exposure time (column name should
        be "expo_time", these are exposure time for each single exposure images in time
        series images), magnitude zero point (column name should be "zero_point", these
        are zero point magnitudes for each single exposure images in time series image),
        coordinates of the object (column name should be "calexp_center"), these are
        the coordinates in (ra, dec), and the band in which the observation is taken
        (column name should be "band").

    :return: Astropy table of injected lenses and exposure information of dp0 data
    """

    final_image = []

    for obs in range(len(exposure_data["obs_time"])):

        exposure_data_obs = exposure_data[obs]

        if exposure_data_obs["band"] not in bands:
            continue

        if "bkg_noise" in exposure_data_obs.keys():
            std_gaussian_noise = exposure_data_obs["bkg_noise"]
        else:
            std_gaussian_noise = None

        lens_images = lens_image(
            lens_class,
            band=exposure_data_obs["band"],
            mag_zero_point=exposure_data_obs["zero_point"],
            num_pix=num_pix,
            psf_kernel=exposure_data_obs["psf_kernel"],
            transform_pix2angle=transform_pix2angle,
            exposure_time=exposure_data_obs["expo_time"],
            t_obs=exposure_data_obs["obs_time"],
            std_gaussian_noise=std_gaussian_noise
        )

        final_image.append(lens_images)

    lens_col = Column(name="lens", data=final_image)
    final_image_col = Column(name="injected_lens", data=final_image)
    
    # Create a new Table with only the bands of interest
    mask = np.isin(exposure_data['band'], bands)
    exposure_data_new = exposure_data[mask]
    exposure_data_new.add_columns([lens_col, final_image_col])
    return exposure_data_new

In [ ]:
N = 100

ra_points = coord.Angle(np.random.uniform(low=0, high=360, size=N) * u.degree)
ra_points = ra_points.wrap_at(180*u.degree)
dec_points = np.arcsin(2 * np.random.uniform(size=N) - 1) / np.pi * 180
dec_points = coord.Angle(dec_points * u.degree)

In [ ]:
exposure_data = opsim_time_series_images_data2(ra_points, dec_points, "baseline_v3.0_10yrs",
                                              MJD_min=60000, MJD_max=60300)
exposure_data

In [ ]:
from slsim.image_simulation import (
    sharp_image,
    lens_image,
    lens_image_series,
)
from astropy.table import Column

index = 1
bands = ['g','r','i','z']
num_pix = 200
transform_pix2angle = np.array([[0.2, 0], [0, 0.2]])

images = opsim_variable_lens_injection(lens_class, bands, num_pix, transform_pix2angle, exposure_data[index])
images

In [ ]:
print("band: ", images['band'][10])
plt.imshow(images['injected_lens'][10])
plt.xlim(75, 125)
plt.ylim(75, 125)

In [ ]:
np.array(exposure_data[0]["band"]) in np.array(bands)

In [ ]:
len(exposure_data[0])

### Notes

Can I make a separate function to calculate the zero point?

calexp_center just contains the object coordinates so is currently a lot of duplicates for each epoch. Maybe there's a more efficient way to save it (or maybe it doesn't matter). 

'injected_lens' now contains the same as 'lens', because there is no background image

In [ ]:
# Next steps: test it out for lensed SN (see the other notebook, try to recreate that with the opsim class)
# do the images make sense with the input?